In [3]:
import pandas as pd

In [25]:
data = pd.read_csv('halo_data.csv')

In [87]:
data['kd_spread'] = data['Kills'] - data['Deaths']
data['kd_ratio'] = data['Kills']/data['Deaths']
data['win_ind'] = 0
data['win_ind'].loc[data['Result'] == 'Win'] = 1

In [157]:
# Overall Player Stat Generation

overall_stats = pd.DataFrame(data.groupby('Player').agg(
    total_games = pd.NamedAgg(column = 'Game Number', aggfunc = 'count')
    ,total_wins = pd.NamedAgg(column = 'win_ind', aggfunc = sum)
    ,total_kills = pd.NamedAgg(column = 'Kills', aggfunc = sum)
    ,total_assists = pd.NamedAgg(column = 'Assists', aggfunc = sum)
    ,total_deaths = pd.NamedAgg(column = 'Deaths', aggfunc = sum)
)).reset_index()

overall_stats['Map'] = 'Overall'

map_stats = pd.DataFrame(data.groupby(['Player', 'Map']).agg(
    total_games = pd.NamedAgg(column = 'Game Number', aggfunc = 'count')
    ,total_wins = pd.NamedAgg(column = 'win_ind', aggfunc = sum)
    ,total_kills = pd.NamedAgg(column = 'Kills', aggfunc = sum)
    ,total_assists = pd.NamedAgg(column = 'Assists', aggfunc = sum)
    ,total_deaths = pd.NamedAgg(column = 'Deaths', aggfunc = sum)
)).reset_index()

cum_all_stats = pd.concat([overall_stats, map_stats], sort=False)

cum_all_stats['win_percentage'] = round(cum_all_stats['total_wins'] / cum_all_stats['total_games'] * 100, 1)
cum_all_stats['k/d ratio'] = round(cum_all_stats['total_kills'] / cum_all_stats['total_deaths'], 2)
cum_all_stats['k/d spread'] = cum_all_stats['total_kills'] - cum_all_stats['total_deaths']
cum_all_stats['avg kills per game'] = round(cum_all_stats['total_kills'] / cum_all_stats['total_games'], 1)
cum_all_stats['avg assists per game'] = round(cum_all_stats['total_assists'] / cum_all_stats['total_games'], 1)
cum_all_stats['avg deaths per game'] = round(cum_all_stats['total_deaths'] / cum_all_stats['total_games'], 1)

cum_overall_total_games = cum_all_stats.loc[cum_all_stats['Map'] == 'Overall'][['Player','total_games']].sort_values('total_games', ascending = False).reset_index(drop = True)
cum_overall_win = cum_all_stats.loc[cum_all_stats['Map'] == 'Overall'][['Player','total_wins']].sort_values('total_wins', ascending = False).reset_index(drop = True)
cum_overall_win_percentage = cum_all_stats.loc[cum_all_stats['Map'] == 'Overall'][['Player','win_percentage']].sort_values('win_percentage', ascending = False).reset_index(drop = True)
cum_overall_avg_stats = cum_all_stats.loc[cum_all_stats['Map'] == 'Overall'][['Player', 'avg kills per game', 'avg assists per game', 'avg deaths per game']].sort_values('avg kills per game', ascending = False).reset_index(drop = True)
cum_overall_kdratio = cum_all_stats.loc[cum_all_stats['Map'] == 'Overall'][['Player','k/d ratio']].sort_values('k/d ratio', ascending = False).reset_index(drop = True)
cum_overall_cumulative = cum_all_stats.loc[cum_all_stats['Map'] == 'Overall'][['Player', 'total_kills', 'total_assists', 'total_deaths', 'k/d spread']].sort_values('total_kills', ascending = False).reset_index(drop = True)

cum_overall_dfs = [cum_overall_total_games, cum_overall_win, cum_overall_win_percentage, cum_overall_avg_stats, cum_overall_cumulative, cum_overall_kdratio]

for i in cum_overall_dfs:
    i.index += 1



In [162]:
# Recent Player Stat Generation

recent_date = data.Date.max()

recent_overall_stats = pd.DataFrame(data.loc[data['Date'] == recent_date].groupby('Player').agg(
    total_games = pd.NamedAgg(column = 'Game Number', aggfunc = 'count')
    ,total_wins = pd.NamedAgg(column = 'win_ind', aggfunc = sum)
    ,total_kills = pd.NamedAgg(column = 'Kills', aggfunc = sum)
    ,total_assists = pd.NamedAgg(column = 'Assists', aggfunc = sum)
    ,total_deaths = pd.NamedAgg(column = 'Deaths', aggfunc = sum)
)).reset_index()

recent_overall_stats['Map'] = 'Overall'

recent_map_stats = pd.DataFrame(data.loc[data['Date'] == recent_date].groupby(['Player', 'Map']).agg(
    total_games = pd.NamedAgg(column = 'Game Number', aggfunc = 'count')
    ,total_wins = pd.NamedAgg(column = 'win_ind', aggfunc = sum)
    ,total_kills = pd.NamedAgg(column = 'Kills', aggfunc = sum)
    ,total_assists = pd.NamedAgg(column = 'Assists', aggfunc = sum)
    ,total_deaths = pd.NamedAgg(column = 'Deaths', aggfunc = sum)
)).reset_index()

rec_all_stats = pd.concat([recent_overall_stats, recent_map_stats], sort=False)

rec_all_stats['win_percentage'] = round(rec_all_stats['total_wins'] / rec_all_stats['total_games'] * 100, 1)
rec_all_stats['k/d ratio'] = round(rec_all_stats['total_kills'] / rec_all_stats['total_deaths'], 2)
rec_all_stats['k/d spread'] = rec_all_stats['total_kills'] - rec_all_stats['total_deaths']
rec_all_stats['avg kills per game'] = round(rec_all_stats['total_kills'] / rec_all_stats['total_games'], 1)
rec_all_stats['avg assists per game'] = round(rec_all_stats['total_assists'] / rec_all_stats['total_games'], 1)
rec_all_stats['avg deaths per game'] = round(rec_all_stats['total_deaths'] / rec_all_stats['total_games'], 1)

rec_overall_total_games = rec_all_stats.loc[rec_all_stats['Map'] == 'Overall'][['Player','total_games']].sort_values('total_games', ascending = False).reset_index(drop = True)
rec_overall_win = rec_all_stats.loc[rec_all_stats['Map'] == 'Overall'][['Player','total_wins']].sort_values('total_wins', ascending = False).reset_index(drop = True)
rec_overall_win_percentage = rec_all_stats.loc[rec_all_stats['Map'] == 'Overall'][['Player','win_percentage']].sort_values('win_percentage', ascending = False).reset_index(drop = True)
rec_overall_avg_stats = rec_all_stats.loc[rec_all_stats['Map'] == 'Overall'][['Player', 'avg kills per game', 'avg assists per game', 'avg deaths per game']].sort_values('avg kills per game', ascending = False).reset_index(drop = True)
rec_overall_kdratio = rec_all_stats.loc[rec_all_stats['Map'] == 'Overall'][['Player','k/d ratio']].sort_values('k/d ratio', ascending = False).reset_index(drop = True)
rec_overall_cumulative = rec_all_stats.loc[rec_all_stats['Map'] == 'Overall'][['Player', 'total_kills', 'total_assists', 'total_deaths', 'k/d spread']].sort_values('total_kills', ascending = False).reset_index(drop = True)

rec_overall_dfs = [rec_overall_total_games, rec_overall_win, rec_overall_win_percentage, rec_overall_avg_stats, rec_overall_cumulative, rec_overall_kdratio]

for i in rec_overall_dfs:
    i.index += 1



In [166]:
# Write Dataframes to Excel

writer = pd.ExcelWriter('THE Halo Statline.xlsx', engine = 'xlsxwriter')
workbook = writer.book

worksheet = workbook.add_worksheet('All Maps Stats')
writer.sheets['All Maps Stats'] = worksheet

worksheet.write(0, 0, 'Overall Stats -- Cumulative')
worksheet.write(1, 1, 'Total Games')
cum_overall_total_games.to_excel(writer, sheet_name = 'All Maps Stats', startrow = 2, startcol = 0)
worksheet.write(1, 5, 'Total Wins')
cum_overall_win.to_excel(writer, sheet_name = 'All Maps Stats', startrow = 2, startcol = 4)
worksheet.write(1, 9, 'Win Percentage')
cum_overall_win_percentage.to_excel(writer, sheet_name = 'All Maps Stats', startrow = 2, startcol = 8)
worksheet.write(1, 13, 'K/D Ratio')
cum_overall_kdratio.to_excel(writer, sheet_name = 'All Maps Stats', startrow = 2, startcol = 12)
worksheet.write(1, 17, 'Average Stats Per Game')
cum_overall_avg_stats.to_excel(writer, sheet_name = 'All Maps Stats', startrow = 2, startcol = 16)
worksheet.write(1, 23, 'Cumulative Stats')
cum_overall_cumulative.to_excel(writer, sheet_name = 'All Maps Stats', startrow = 2, startcol = 22)

vertical_buffer = len(cum_overall_total_games) + 4

worksheet.write(0 + vertical_buffer, 0, 'Overall Stats -- Recent Halo Night On ' + recent_date)
worksheet.write(1+ vertical_buffer, 1, 'Total Games')
rec_overall_total_games.to_excel(writer, sheet_name = 'All Maps Stats', startrow = 2 + vertical_buffer, startcol = 0)
worksheet.write(1+ vertical_buffer, 5, 'Total Wins')
rec_overall_win.to_excel(writer, sheet_name = 'All Maps Stats', startrow = 2 + vertical_buffer, startcol = 4)
worksheet.write(1+ vertical_buffer, 9, 'Win Percentage')
rec_overall_win_percentage.to_excel(writer, sheet_name = 'All Maps Stats', startrow = 2 + vertical_buffer, startcol = 8)
worksheet.write(1+ vertical_buffer, 13, 'K/D Ratio')
rec_overall_kdratio.to_excel(writer, sheet_name = 'All Maps Stats', startrow = 2 + vertical_buffer, startcol = 12)
worksheet.write(1+ vertical_buffer, 17, 'Average Stats Per Game')
rec_overall_avg_stats.to_excel(writer, sheet_name = 'All Maps Stats', startrow = 2 + vertical_buffer, startcol = 16)
worksheet.write(1+ vertical_buffer, 23, 'Cumulative Stats')
rec_overall_cumulative.to_excel(writer, sheet_name = 'All Maps Stats', startrow = 2 + vertical_buffer, startcol = 22)


writer.save()

In [165]:
x

8